In [1]:
from multiprocessing import Pool
import tqdm
import pickle
from pathlib import Path
from contextlib import closing
from rdkit import Chem

In [2]:
import sys
sys.path.append("../")

In [3]:
from phenixml.fragmentation.fragmenter_restraints import BondFragmenter, AngleFragmenter

In [4]:
%%time
filtered_dir = Path("/net/cci/cschlick/Filtered_COD3/")
err_files = [path for path in filtered_dir.glob("**/*") if path.suffix == ".err"]

CPU times: user 3.29 s, sys: 8.33 s, total: 11.6 s
Wall time: 58.1 s


In [5]:
%%time
success_converted = []
for err_file in err_files:
  with err_file.open("r") as fh:
    lines = fh.readlines()
    if len(lines)==1 and "1 molecule converted" in lines[0]:
      success_converted.append(Path(err_file.parent,err_file.stem+".mol2"))
print("Success:",len(success_converted))

Success: 74249
CPU times: user 2.9 s, sys: 3.04 s, total: 5.94 s
Wall time: 40.7 s


In [6]:
# elements and parameters
covalent_organic = ["O","C","H","N","P","S","Cl","B","F","I","Br"]
params = {'elements_considered': covalent_organic}

In [7]:
def worker(mol2_file):

  rdmol = Chem.MolFromMol2File(mol2_file.as_posix(),removeHs=False)
  results = {"filepath":mol2_file,"rdmol":rdmol}
  return results

In [8]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*') 

work = success_converted
with closing(Pool(processes=32)) as pool:
  results = []
  for result in tqdm.tqdm(pool.map(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 74249/74249 [00:11<00:00, 6714.06it/s]


In [9]:
success_initialized = []
failed_initialized = []
for result in results:
  if result["rdmol"] is not None:
    success_initialized.append(result)
  else:
    failed_initialized.append(result)
print("Success initialized:",len(success_initialized))
print("Failed initialized:",len(failed_initialized))

Success initialized: 63018
Failed initialized: 11231


In [10]:
angle_fragmenter = AngleFragmenter()
bond_fragmenter = BondFragmenter()

In [11]:
def worker(work_dict):
  rdmol = work_dict["rdmol"]
  angle_fragments = angle_fragmenter.fragment(rdmol)
  angle_fragments = [frag for frag in angle_fragments if "H" not in frag.atom_symbols]

  
  bond_fragments = bond_fragmenter.fragment(rdmol)
  bond_fragments = [frag for frag in bond_fragments if "H" not in frag.atom_symbols]
  
  for frag in angle_fragments:
    frag.properties["filepath"]=work_dict["filepath"]
    conf = frag.rdmol.GetConformer()
    i,j,k = frag.atom_indices
    angle_deg = Chem.rdMolTransforms.GetAngleDeg(conf,i,j,k)
    frag.properties["angle_deg"] = angle_deg

    
  for frag in bond_fragments:
    frag.properties["filepath"]=work_dict["filepath"]
    
    conf = frag.rdmol.GetConformer()
    i,j = frag.atom_indices
    bond_length = Chem.rdMolTransforms.GetBondLength(conf,i,j)
    frag.properties["bond_length"] = bond_length
    
  work_dict["angle_fragments"]=angle_fragments
  work_dict["bond_fragments"]=bond_fragments
  return work_dict

In [12]:
work = success_initialized
with closing(Pool(processes=32)) as pool:
  results = []
  for result in tqdm.tqdm(pool.map(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 63018/63018 [01:00<00:00, 1046.73it/s]


In [13]:
angle_fragments = []
bond_fragments = []
for result in results:
  angle_fragments+=result["angle_fragments"]
  bond_fragments+=result["bond_fragments"]

In [20]:
%%time
fragments_path = Path("/net/cci/cschlick/Filtered_COD3/fragments.pkl")
with fragments_path.open("wb") as fh:
  pickle.dump(results,fh)

CPU times: user 14.3 s, sys: 1.93 s, total: 16.3 s
Wall time: 19.3 s


## Stop

In [36]:
rdmols = [frag.rdmol for frag in frags]
assert len(set(rdmols))==1
rdmol = rdmols[0]
interaction_mols = [rdmol]
target_fragments = frags

In [58]:
frags_midpoints = np.array([target.xyz_fragment.mean(axis=1) for target in target_fragments])
assert(frags_midpoints.shape[1]==3)
target_xyz = frags_midpoints
interaction_xyz = np.vstack([rdmol.GetConformer().GetPositions() for rdmol in interaction_mols])
interaction_elements = np.concatenate([[a.GetSymbol() for a in rdmol.GetAtoms()] for rdmol in interaction_mols])

In [85]:
# elements and parameters
covalent_organic = ["O","C","H","N","P","S","Cl","B","F","I","Br"]
metals = ["Na","K","Ca","Fe","Mn","Zn","Mg","Cu","Co"]

params = {'radial_cutoff': 4.6,
 'radial_nu': 32,
 'radial_probes': [0.7,
                  1.4,
                  1.9,
                  2.4,
                  3.2,
                  3.8,
                  4.4],
 'angular_cutoff': 3.1,
 'angular_nu': 4,
 'angular_probes': [0.0, 1.57, 3.14, 4.71],
 'angular_radial_probes': [0.7,1.4,1.9,2.4],
 'angular_zeta': 8,
 'min_probed_value': 0.0,
 'exclude_hydrogens': False,
 'elements_considered': covalent_organic}

ModuleNotFoundError: No module named 'featurizer_base'

In [99]:
aev = ANIVector(target_xyz,interaction_xyz,interaction_elements,params)

In [98]:
aev.featurize()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [110]:
feat = MorganFeaturizer()

In [112]:
ret = feat.featurize(frags[0])

In [114]:
np.array(ret)

array([0, 0, 0, ..., 0, 0, 0])

In [102]:
import rdkit

In [105]:
Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect??

Docstring:
GetMorganFingerprintAsBitVect( (Mol)mol, (int)radius [, (int)nBits=2048 [, (AtomPairsParameters)invariants=[] [, (AtomPairsParameters)fromAtoms=[] [, (bool)useChirality=False [, (bool)useBondTypes=True [, (bool)useFeatures=False [, (AtomPairsParameters)bitInfo=None [, (bool)includeRedundantEnvironments=False]]]]]]]]) -> ExplicitBitVect :
    Returns a Morgan fingerprint for a molecule as a bit vector

    C++ signature :
        ExplicitBitVect* GetMorganFingerprintAsBitVect(RDKit::ROMol,unsigned int [,unsigned int=2048 [,boost::python::api::object=[] [,boost::python::api::object=[] [,bool=False [,bool=True [,bool=False [,boost::python::api::object=None [,bool=False]]]]]]]])
Type:      function
